In [ ]:
                                                    # Quantum information Reversibility
                                            # Inspired by: Black hole information loss problem

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from scipy.special import rel_entr
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
from datetime import datetime
import os
from qiskit.quantum_info import state_fidelity

n_qubits = 6                                       # I tried upto 24 qubits from 6 and i found something interesting for which you have to refer to the Readme file
shots = 1024
simulator = AerSimulator()
os.makedirs("results", exist_ok=True)


def initial_state():
    # Creating an initial entangled state (0, 1) (2, 3) (4, 5)
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(0, n_qubits, 2):
        qc.h(i)
        qc.cx(i, i + 1)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc


def noise(qc, rx_angle=np.pi/3, rz_angle=np.pi/4):
    # Applying noise: rotations and entangling gates for all gates
    for i in range(n_qubits):
        qc.rx(rx_angle, i)
        qc.rz(rz_angle, i)

     # Information Scrambling here

    qc.cx(0, 2)
    qc.cz(1, 3)
    qc.cx(4, 5)


def post_noise():
    # Circuit after adding noise to the initial state
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i in range(0, n_qubits, 2):
        qc.h(i)
        qc.cx(i, i + 1)

    noise(qc)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc


def reversed_state():
    # Reversing the noise and entanglement applied in the post state
    qc = QuantumCircuit(n_qubits, n_qubits)

    # Initial entanglement
    for i in range(0, n_qubits, 2):
        qc.h(i)
        qc.cx(i, i + 1)

    # Apply noise again
    noise(qc)

    # Reversing the noise
    qc.cx(0, 2)
    qc.cz(1, 3)
    qc.cx(4, 5)

    for i in range(n_qubits):
        qc.rz(-np.pi/4, i)
        qc.rx(-np.pi/3, i)

    # Reversing the entangled state
    for i in range(0, n_qubits, 2):
        qc.cx(i, i + 1)
        qc.h(i)

    qc.measure(range(n_qubits), range(n_qubits))
    return qc


def run_circuit(qc, label):
    transpiled = transpile(qc, simulator)
    result = simulator.run(transpiled, shots=shots).result()
    counts = result.get_counts()
    print(f"\n[{label}] Counts:")
    print(counts)
    return counts


def generate_all_states(n):
    return [''.join(bits) for bits in product('01', repeat=n)]


def get_probabilities(counts, all_states):
    total = sum(counts.values())
    return np.array([counts.get(state, 0) / total for state in all_states])


def kl_divergence(p_counts, q_counts, n):
    all_states = generate_all_states(n)
    p_probs = get_probabilities(p_counts, all_states) + 1e-10
    q_probs = get_probabilities(q_counts, all_states) + 1e-10
    return np.sum(rel_entr(p_probs, q_probs))


if __name__ == "__main__":

    #  Pre-noise state 
    pre_qc = initial_state()
    pre_counts = run_circuit(pre_qc, "Pre-noise (Initial State)")

    # Post-noise state 
    post_qc = post_noise()
    post_counts = run_circuit(post_qc, "Post-noise (Inside Black Hole State)")

    # Reversed state 
    reversed_qc = reversed_state()
    reverse_counts = run_circuit(reversed_qc, "Reversed State (After Time Reversal)")

    # KL Divergence 
    kl_post = kl_divergence(pre_counts, post_counts, n_qubits)
    kl_reversed = kl_divergence(pre_counts, reverse_counts, n_qubits)

    print("\n[KL Divergence Results]")
    print(f"KL (Pre vs Post)     : {kl_post:.6f} → Expected to be HIGH")
    print(f"KL (Pre vs Reversed) : {kl_reversed:.6f} → Should be LOW if reversal worked")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"results/entropy_plot_{timestamp}.png"

    plot_histogram(
        [pre_counts, post_counts, reverse_counts],  # I dont know why but the histogram always outputs a blank page
        legend=["Pre", "Post", "Reversed"],
        color=['#FF5733', '#33FF57', '#3357FF'],
        number_to_keep=20,
        figsize=(12, 6)
    )

    plt.title("Probability States")
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.show()

    print(f"Results saved to: {filename}")


Relative Entropy (KL Divergence): 0.00042918045994144727
